## Test all the things!
* Write/update transactions to get confirmation of success/failure
* Custom indexes to make querying by properties possible

In [33]:
from google.cloud import datastore
from datetime import datetime

PROJECT = 'reliable-realm-222318'

In [34]:
client = datastore.Client(project=PROJECT)

### Transactions

In [35]:
with client.transaction() as xact:
    key = client.key('Task')
    item = datastore.Entity(key=key)
    item['description'] = 'return a value maybe?'
    client.put(item)
    mutations = xact.mutations

In [45]:
mut = mutations[0]

In [53]:
from google.protobuf.json_format import MessageToDict

In [56]:
mut_dict = MessageToDict(mut)

## Querying

In [287]:
key = client.key('PlanetScenes', '20180601_182755_0f33')
result = client.get(key)

result.keys()

dict_keys(['_permissions', 'id', 'properties', 'assets', '_links', 'geometry', 'visual_downloaded', 'visual_annotated', 'type'])

In [241]:
result['images']

[<Entity {'visual': 'gs://pipeline/full/PSScene3Band_20180601_182755_0f33_visual.tiff'}>]

In [332]:
query = client.query(kind='PlanetScenes')
# query.projection = ['images']
# query.projection = ['assets.visual.status', 'assets.visual.location']
# query.projection = ['visual_downloaded']
# query.order = ['images.visual']
query.order = ['-properties.acquired']
query.add_filter('visual_downloaded', '=', True)
query.add_filter('visual_annotated', '=', False)
# query.projection = ['images.visual']
# query.distinct_on = ['visual_downloaded']
result = list(query.fetch(limit=50))

FailedPrecondition: 400 no matching index found. recommended index is:
- kind: PlanetScenes
  properties:
  - name: visual_annotated
  - name: visual_downloaded
  - name: properties.acquired
    direction: desc


In [331]:
result[0]['properties']['acquired']

'2018-06-01T18:22:49.091758Z'

In [121]:
import requests
URL = 'https://datastore.googleapis.com/v1/projects/{}/indexes'.format(PROJECT)
requests.get(URL).content

b'{\n  "error": {\n    "code": 401,\n    "message": "Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",\n    "status": "UNAUTHENTICATED"\n  }\n}\n'

In [ ]:
import google

In [78]:
[r['properties.acquired'] for r in result]

['2018-07-14T18:23:29.181777Z',
 '2018-07-14T18:21:55.152113Z',
 '2018-07-14T18:04:27.29376Z']

In [206]:
query = client.query(kind='PlanetScenes')
query.projection = ['images.visual']
result = list(query.fetch(limit=10))

In [207]:
result

[]

## Cursor

In [297]:
query = client.query(kind='PlanetScenes')
query.add_filter('visual_downloaded', '=', True)
query.add_filter('visual_annotated', '=', False)
result = list(query.fetch(limit=10))

In [304]:
def get_one_page_of_tasks(cursor=None):
    query = client.query(kind='PlanetScenes')
    query.add_filter('visual_downloaded', '=', True)
    query.add_filter('visual_annotated', '=', False)
    query.order = ['-properties.acquired']

    query_iter = query.fetch(start_cursor=cursor, limit=5)
    page = next(query_iter.pages)

    entities = list(page)
    next_cursor = query_iter.next_page_token

    return entities, next_cursor

In [306]:
results, next_cursor = get_one_page_of_tasks(cursor=None)

In [307]:
len(results), next_cursor

(5,
 b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDJfMTgyMzA1XzEwMmUMGAAgAA==')

In [311]:
next_cursor = None
for i in range(8):
    results, next_cursor = get_one_page_of_tasks(cursor=next_cursor)
    print(next_cursor)

b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDJfMTgyMzA1XzEwMmUMGAAgAA=='
b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDJfMjEzNTQxXzBmMDIMGAAgAA=='
b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDRfMTgyMzI0XzBlMGYMGAAgAA=='
None
b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDJfMTgyMzA1XzEwMmUMGAAgAA=='
b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDJfMjEzNTQxXzBmMDIMGAAgAA=='
b'CkcSQWoXc35yZWxpYWJsZS1yZWFsbS0yMjIzMThyJgsSDFBsYW5ldFNjZW5lcyIUMjAxODA2MDRfMTgyMzI0XzBlMGYMGAAgAA=='
None
